In [1]:
from micrograd.nn import MLP

In [6]:
input_vals = [
    [1.0, 2.0, 3.0],
    [3.0, 8.0, 2.0],
    [7, 8, 3],
    [3,5,2],
    [6,5,2]
]

out_vals = [6, 48, 168, 30, 60]
model = MLP(3, [20, 20, 1])

In [3]:
from graphviz import Digraph


def trace(root):
    # builds a set of all nodes and edges in a graph
    nodes, edges = set(), set()

    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)

    build(root)
    return nodes, edges


def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})  # LR = left to right

    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        # for any value in the graph, create a rectangular ('record') node for it
        dot.node(name=uid, label="{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
        if n._op:
            # if this value is a result of some operation, create an op node for it
            dot.node(name=uid + n._op, label=n._op)
            # and connect this node to it
            dot.edge(uid + n._op, uid)

    for n1, n2 in edges:
        # connect n1 to the op node of n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)

    return dot

In [7]:
res = model(input_vals[0])
draw_dot(res)

In [11]:
for i, o in zip(input_vals, out_vals):
    res = model(i)
    loss = res - o
    loss = -loss if loss.data < 0 else loss
    print('LOSS: ' , loss)
    draw_dot(res)
    model.zero_grad()
    loss.backward()
    for p in model.parameters():
        p.data += -0.1 * p.grad

LOSS:  Value(data=0.09705239957524281, grad=0)
LOSS:  Value(data=37.52816341773795, grad=0)
LOSS:  Value(data=156.53925078649146, grad=0)
LOSS:  Value(data=18.09047532714695, grad=0)
LOSS:  Value(data=46.30965813495059, grad=0)
